In [17]:
%matplotlib inline
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np
import traml
import copy

In [2]:
a = traml.Stack()

|   # |   Material | Thickness (nm)   | Type      |
|-----+------------+------------------+-----------|
|   0 |     301005 | --               | substrate |


In [3]:
a.library.search('CdTe')

|     ID | SHELF   | BOOK   | PAGE        | DESCRIPTION                    |
|--------+---------+--------+-------------+--------------------------------|
| 116701 | main    | CdTe   | DeBell-300K | DeBell et al. 1979: n 6-22 µm; |
| 116702 | main    | CdTe   | DeBell-80K  | DeBell et al. 1979: n 6-22 µm; |
| 116703 | main    | CdTe   | DeBell-20K  | DeBell et al. 1979: n 6-22 µm; |
| 116704 | main    | CdTe   | Marple      | Marple 1964: n 0.86-2.5 µm     |
| 116705 | main    | CdTe   | Treharne    | Treharne et al. 2011: n,k 0.3- |


In [4]:
CdS = 114203
CdTe = 116705
CdZnS = 114301

In [5]:
a.library.plot(CdS)

In [6]:
def fun(x, stack, layers, c, delta, TR):
    for i in layers:
          
        
        stack.d(i, x[i-1])
    #stack.d(1,x[0])
    #stack.d(2,x[1])
    for p in x:
        p = np.sqrt(x[0]**2)
    x[0]=np.sqrt(x[0]**2)
    min = 1-stack.average(TR, c, delta, o='a')
    #min = stack.average(TR, c, delta)
    #print min

    return min**10

In [7]:
def minTR(stack, film_range, c, delta, TR='T'):
    print film_range
    x = []
    layers = []
    bnds = []
    for i in range(film_range[0],film_range[1]+1):
        #print i
        x.append(stack.config[i][1])
        layers.append(i)
        bnds.append(stack.config[i][3])
        
    result = minimize(fun, x, args=(stack, layers, c,delta, TR), bounds = bnds,tol=1e-30, method='SLSQP')
    
    #stack.plot()
    print layers
    return result

In [8]:
c = traml.Stack()

peak = 0.6
delta = 0.2
mat2 = 105401 #MgF2
sio2 = 111407 #SiO2
zns = 114801 #ZnS
zno = 112003 #ZnO
mgo = 110902 #MgO
#mat1 = 110104 #Al2O3
zro2 = 112101
stack_list = []

c.add(zno, 500, bnds = (500, 2500))
c.add(CdZnS, 50, bnds=(25, 250))
c.add(CdTe, film_type='absorber')

stack_list.append(copy.deepcopy(c))
R=[]
point = []
point.append(c.jsc())
print point
n = [0]
it = [1]

result = minTR(c, [1,len(c.config)-2], peak, delta, TR='T')
point.append(c.jsc())
n.append(0)
stack_list.append(copy.deepcopy(c))

c.remove()
c.remove()
c.remove()

N = 5
for i in range(0, N):
    if i % 2 == 0:
        print 'even', i
        c.add(zns,20, bnds=(0,100))
    else:
        c.add(sio2,20, bnds=(0,100))
        
    c.add(zno, 500, bnds = (500, 2500))
    c.add(CdZnS, 50, bnds=(25, 250))
    c.add(CdTe, film_type='absorber')
    
    
    result = minTR(c, [1,len(c.config)-2], peak, delta, TR='T')
    R.append(c.average('R',peak,delta))
    n.append(i+1)
    it.append(result.nit)
    stack_list.append(copy.deepcopy(c))
    point.append(c.jsc())
    
    print c.config[i+3][1]
    
   
    c.table()
    print '============================================================================================'
    print c.average('T',peak,delta, o='a')
    
    
    if i !=N-1:
        c.remove()
        c.remove()
        c.remove()
        
    print '============================================================================================'
    
    
    
    
 

|   # |   Material | Thickness (nm)   | Type      |
|-----+------------+------------------+-----------|
|   0 |     301005 | --               | substrate |
|   # |   Material | Thickness (nm)   | Type      |
|-----+------------+------------------+-----------|
|   0 |     301005 | --               | substrate |
|   1 |     112003 | 500              | passive   |
|   # |   Material | Thickness (nm)   | Type      |
|-----+------------+------------------+-----------|
|   0 |     301005 | --               | substrate |
|   1 |     112003 | 500              | passive   |
|   2 |     114301 | 50               | passive   |
|   # |   Material | Thickness (nm)   | Type      |
|-----+------------+------------------+-----------|
|   0 |     301005 | --               | substrate |
|   1 |     112003 | 500              | passive   |
|   2 |     114301 | 50               | passive   |
|   3 |     116705 | 100              | absorber  |
24.1054990673 (ma/cm^2)
[24.105499067290904]
[1, 2]
[1, 2]
25.61

In [9]:
for film in stack_list:
    film.jsc()

24.1054990673 (ma/cm^2)
25.618848626 (ma/cm^2)
26.9143654572 (ma/cm^2)
26.9401457551 (ma/cm^2)
27.0526962424 (ma/cm^2)
27.0052098248 (ma/cm^2)
26.9613191749 (ma/cm^2)


In [20]:
%matplotlib
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111)

l1, = ax.plot(c.x, stack_list[0].get_T(option='a'), '-', color='red', label='T (w/o FSARC)')
l2, = ax.plot(c.x, stack_list[5].get_T(option='a'), '--', color='red', label='T (with FSARC)')
l3, = ax.plot(c.x, stack_list[0].get_R(option='a'), '-', color='blue', label='R (w/o FSARC)')
l4, = ax.plot(c.x, stack_list[5].get_R(option='a'), '--', color='blue', label='R (with FSARC)')


legend = ax.legend(loc='center right', fontsize=16)

#ax.set_xlim([0,14])
#ax.set_xlabel(r'Number of films', fontsize=20)
ax.set_ylabel('T, R', fontsize=20)
ax.set_xlabel('$\lambda$ ($\mu$m)', fontsize=24)
ax.tick_params(labelsize = 20)
ax.set_ylim([0,1])
ax.set_xlim([0.35,0.9])



#plt.plot(a.x, a.get_T(),)
#plt.plot(c.x, c.get_T())
#a.average('R', 0.6,0.3)
#plt.xlim([0.4, 0.9])

Using matplotlib backend: TkAgg


(0.35, 0.9)

In [11]:
c.jsc()

26.9613191749 (ma/cm^2)


26.961319174854822

In [12]:
plt.plot(n, point, '-o')

In [13]:
max(point)

27.052696242385537

In [14]:
fig = plt.figure()

for stack in stack_list:
    stack.jsc()

In [15]:
stack_list[0].config

[[301005, '--', 'substrate'],
 [112003, 500, 'passive', (500, 2500)],
 [114301, 50, 'passive', (25, 250)],
 [116705, 100, 'absorber', (0, 10000)]]